<a href="https://colab.research.google.com/github/Kunal1903/CSOC22-Week0-algorithms/blob/main/fine_tuning_gpt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
import re,string,unicodedata
import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as f

data=pd.read_csv('/content/sample_data/sample_sarcasm.csv');


#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text


In [ ]:
!pip install datasets
from datasets import Dataset
dataset = Dataset.from_pandas(data.rename(columns={"processed_tweets":'text'}))

In [ ]:
dataset["text"][0]

'ka bhagwan bar ka ram lakh lakh pe shukra hai bachho ne ma tuition le aap se nahi mein li.'

In [ ]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/sample_data/temp.txt',
    block_size=128
)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments
# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/sample_data',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)


In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=117, training_loss=5.316947219718216, metrics={'train_runtime': 21.8925, 'train_samples_per_second': 20.966, 'train_steps_per_second': 5.344, 'total_flos': 29983260672000.0, 'train_loss': 5.316947219718216, 'epoch': 3.0})

In [ ]:
model=model.to('cpu')
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model=model,tokenizer=tokenizer)
set_seed(42)
generator("Tu kon hai", max_length=30, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Tu kon hai hua ain raha sathah hai!\nLikhaya bahut na raha aajh'}]

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
#Freezing convolution layer
for param in list(model.transformer.h[0].attn.c_attn.parameters()):
    param.requires_grad=False

list(model.transformer.h[0].attn.c_attn.parameters())

[Parameter containing:
 tensor([[-0.4739, -0.2609, -0.0974,  ...,  0.0513, -0.0584,  0.0248],
         [ 0.0876,  0.1473,  0.2392,  ..., -0.0530, -0.0108, -0.0154],
         [ 0.0038,  0.0696,  0.3662,  ...,  0.1137,  0.0359, -0.0318],
         ...,
         [-0.2598, -0.0159,  0.1996,  ...,  0.0099, -0.0518,  0.0324],
         [ 0.1511,  0.2173,  0.1050,  ...,  0.0294, -0.0429, -0.0474],
         [-0.4095, -0.1921, -0.2402,  ..., -0.0044,  0.0066,  0.0194]]),
 Parameter containing:
 tensor([ 0.4809, -0.5256, -0.4299,  ...,  0.0129, -0.0503,  0.0033])]